In [12]:
import pandas as pd
import numpy as np
from joblib import load
from sklearn.preprocessing import LabelEncoder
from pomegranate import *
from pprint import pprint

In [13]:
df_in = pd.read_csv('../../data_v3/hr_il_tagged.csv')

In [14]:
df = df_in[["frame.number", "event_with_roles", "request_method_call", "starting_frame_number", "selective_filter_data",
            "origin_selective_filter_data", "stream_index", "real_activity_action", ]]


In [15]:
df_gt = pd.read_csv("../../data_v3/hr_ground_truth.csv")

In [16]:
df_gt

,activity_name,start,end,actual_end,overlapping_activities,bp_id,overlapping_bps,classification,Multi Class Classification,Single Class Classification,probability
0,GenerateJobApplicationActivity,17,356,325,[],1224,[],GenerateJobApplicationActivity,TRUE,True,0.994192
1,ResumeReviewActivity,356,1212,1192,[],1224,[],ResumeReviewActivity,TRUE,True,0.993149
2,GenerateJobApplicationActivity,1212,1582,1520,[],1225,[],GenerateJobApplicationActivity,TRUE,True,0.994192
3,ResumeReviewActivity,1582,2354,2336,[],1225,[],ResumeReviewActivity,TRUE,True,0.993329
4,GenerateJobApplicationActivity,2354,2708,2664,[],1226,[],GenerateJobApplicationActivity,TRUE,True,0.994192
5,ResumeReviewActivity,2708,4467,4461,['ScheduleAnInterviewActivityCall'],1226,[1255],"ResumeReviewActivity, ScheduleAnInterviewActiv...",TRUE,True,0.211112
6,ScheduleAnInterviewActivityCall,3057,4881,4871,"['ResumeReviewActivity', 'GenerateJobApplicati...",1225,"[1226,1227]","ResumeReviewActivity, ScheduleAnInterviewActiv...",TRUE,True,0.021792
7,GenerateJobApplicationActivity,4467,4939,4881,['ScheduleAnInterviewActivityCall'],1227,[1226],GenerateJobApplicationActivity,FALSE,True,0.986619
8,ResumeReviewActivity,4939,6205,6164,['PerformAnInterviewCall'],1227,[1225],ResumeReviewActivity,FALSE,True,0.839890
9,PerformAnInterviewCall,5606,6879,6859,['ResumeReviewActivity'],1225,[1227],PerformAnInterviewCall,FALSE,True,0.962106


In [17]:
df["ele_counts"] = df.groupby("real_activity_action").cumcount() + 1
df["start_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"] == "Activity Start" else 0,
                              axis=1).replace(0, None).ffill()
df["end_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"] == "Activity End" else 0,
                            axis=1).replace(0, None).ffill().fillna(0).astype(int)
df["open_activities"] = df["start_counts"] - df["end_counts"]

/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_91603/2128266899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ele_counts"] = df.groupby("real_activity_action").cumcount() + 1
/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_91603/2128266899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["start_counts"] = df.apply(lambda row: row["ele_counts"] if row["real_activity_action"] == "Activity Start" else 0,
/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_91603/212

# Markov Chains for Event Classification

In [18]:
# Prepare training data
# Chain Features 5 Events backward 
df_train = pd.read_csv('../../data_v3/hr_extended_features.csv')

# Form Sequences by Grouping
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

cols = ["event_with_roles","selective_filter_data","origin_selective_filter_data","SequenceNumber"]
seq_numbers = df_train["SequenceNumber"]
df_train = df_train[cols].fillna("")

df_train["joined"] = df_train["event_with_roles"] + df_train["selective_filter_data"]


In [19]:
# Label Encode DF 
le1 = LabelEncoder()
le2 = LabelEncoder()
df_train["event_with_roles_LE"] = le1.fit_transform(df_train["event_with_roles"])
df_train["selective_filter_data_LE"] = le2.fit_transform(df_train["selective_filter_data"])
df_train["origin_selective_filter_data_LE"] = le2.transform(df_train["origin_selective_filter_data"])

le3 = LabelEncoder()
df_train["joined_LE"] = le3.fit_transform(df_train["joined"])

In [20]:
# Divides dataframe into arrays according to to Sequence Data Indicator
def sequence_by_activities(data, seq_data): 
    return [data[seq_data == i].values for i in range(seq_data.max())]

cols_LE = ["event_with_roles_LE","selective_filter_data_LE","origin_selective_filter_data_LE"]
cols = ["event_with_roles","selective_filter_data","origin_selective_filter_data"]

data_LE = sequence_by_activities(df_train[cols_LE], df_train["SequenceNumber"])

data_joined_LE = sequence_by_activities(df_train["joined_LE"], df_train["SequenceNumber"])

data_joined = sequence_by_activities(df_train["joined"], df_train["SequenceNumber"])

In [21]:
# Convert each array to a tuple and create a set of tuples
array_set = set(tuple(arr) for arr in data_joined_LE)

# Convert the set of tuples back to a list of NumPy arrays
unique_array_list = [np.array(arr) for arr in array_set]

print(f"Reduced the number of sequences from {len(data_joined_LE)} to {len(unique_array_list)} unique ones ")

Reduced the number of sequences from 932 to 26 unique ones 


In [22]:
from numpy.lib.stride_tricks import sliding_window_view
# form sliding window sequences of Size N for Training Data 
windows = [np.concatenate([sliding_window_view(seq, i) for seq in unique_array_list], axis = 0) for i in range(6)]

In [23]:
#mc = MarkovChain.from_samples(unique_array_list, k=3)

# Event Loop

In [28]:
df["selective_filter_data"].fillna("", inplace=True)

/var/folders/qt/gt7q1tn978x_02pv47b2lrvm0000gn/T/ipykernel_91603/1213319550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["selective_filter_data"].fillna("", inplace=True)


In [29]:
df

,frame.number,event_with_roles,request_method_call,starting_frame_number,selective_filter_data,origin_selective_filter_data,stream_index,real_activity_action,ele_counts,start_counts,end_counts,open_activities
0,17,End Point (HR Manager)->Odoo Application: [Htt...,version,NaN,version,NaN,1,Activity Start,1,1,0,1
1,49,Odoo Application->End Point (HR Manager): [Htt...,server_version,17.0,,version,1,NoAction,1,1,0,1
2,57,End Point (HR Manager)->Odoo Application: [Htt...,authenticate,NaN,,NaN,5,NoAction,2,1,0,1
3,101,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,,NaN,8,NoAction,3,1,0,1
4,110,Odoo Application->End Point (HR Manager): [Htt...,6,57.0,IsNumber,NaN,5,NoAction,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1293,28267,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,,NaN,12,NoAction,1221,37,36,1
1294,28304,Odoo Application->End Point (Recruiting manage...,1,27852.0,IsNumber,hr.applicant_write_stage_id_8,535,NoAction,1222,37,36,1
1295,28312,End Point (Recruiting manager)->Odoo Applicati...,execute_kw,NaN,hr.applicant_archive_applicant,NaN,538,NoAction,1223,37,36,1
1296,28377,Odoo Application->db Server/Mail Server: [Pgsq...,NaN,NaN,,NaN,8,NoAction,1224,37,36,1


In [25]:
records = df.to_dict('records')
crf = load('activity_model_v2.joblib')

In [38]:
def search_stack(frame_number):
    for index, stack in enumerate(stacks):
        for event in stack:
            if event["frame.number"] == frame_number:
                return index
    return -1


def seq2features(seq, bw):
    # seq [0] = Position in Sequence
    # seq [0,4] = Feature 4 an Position 0 
    return [event2features(seq, i, bw) for i in range(len(seq))]


def event2features(seq, i, bw):
    features = {"bias": 1.0}

    # event features 
    features.update({
        f"0:{key}": value for key, value in seq[i].items()
    })

    # backward features    
    # i: position of event in sequence 
    # j: feature window position from event
    # k: position of related event [-j] in feature window
    for j in range(1, bw + 1):
        k = i - j
        features.update({
            f"-{j}:{key}": seq[k][key] if k >= 0 else "NoMessage"
            for key in seq[i]
        })

    return features

def dict_mean(dict_list):
    mean_dict = {}
    for key in dict_list[0].keys():
        mean_dict[key] = np.mean([d[key] for d in dict_list], axis=0)
    return mean_dict         

def get_improvement(mean, mean_new): 
   # get best key in old data
   #  max_key, max_value = max(mean.items(), key=lambda x: x[1])
   #  
   #  # get best key in new data 
   #  max_value_new = mean_new[max_key]
   #  
   #  # improvement 
   #  improvement = max_value_new - max_value
   #  
   # # print(max_key,f"{improvement:.4f}",max_value,"->", max_value_new)
   # print(f"{max_key} {improvement:.4f} {max_value:.4f} -> {max_value_new:.4f}")
   
   max_value = max(mean_new.values())

   # get differences in prediction
   diff_dict = {k: mean_new[k] - mean[k] for k in mean}
   
   # get absolute improvement in regard to information value
   delta = sum(abs(value) for value in diff_dict.values())
   res = delta + max_value
   
   pprint([f"{k[:3]}, {mean[k]:.4f}, {mean_new[k]:.4f}, {diff_dict[k]:.4f}" for k in mean])
   print(f"{delta:.4f}, {res:.4f}")
   
   return delta
    


def search_window_for_sequence(seq): 
    """
    Check for pattern matches with the training data and return the count
    :param seq: array_like
                sequence of events
            
    :return: number of occurences of seq in training data
    """
    return np.sum(np.all(windows[len(seq)] == seq, axis = 1))
    

def event_to_string(event): 
    """
    Transforms an event to string representation based on its attributes *event_with_roles* and *selective_filter_data*
    :param event: 
    :return: Concatenated string of *event_with_roles* and *selective_filter_data* attributes
    """
    return event["event_with_roles"]+event["selective_filter_data"]

def classify_by_train_sequences(event,n): 
    
    print("event:",event["frame.number"])
    
    #print([len(stack) for stack in stacks])
    # search for existing stacks in training data 
    sequences = [le3.transform([ event_to_string(e) for e in stack]+[event_to_string(event)]) for stack in stacks]
    
    print([seq[-n:].tolist() for seq in sequences])
    
    
    # loop to max 2 elements down
    for i in range(n, 1, -1):   
        res = [search_window_for_sequence(seq[-i:]) for seq in sequences]
        
        max_res = max(res)
        max_res_count = res.count(max_res)
        idx = np.argmax(res)
        
        if max_res > 0: 
            print("res:",res, max_res, max_res_count, "->", idx)
            return idx

    # for elements that are not matchable based on 2 sequences we fall back to stream index
    idx = search_stream_index(event)
    print("STREAM INDEX", idx)
    return 0


def classify_event_by_information_gain(event):
    # build sequences of each event stack with the new event appended
    sequences = [seq2features(stack + [event], bw=5) for stack in stacks]
    predictions = crf.predict_marginals([seq[:-1] for seq in sequences])
    predictions_new = crf.predict_marginals(sequences)
    print([len(stack) for stack in stacks])

    mean = [dict_mean(pred) for pred in predictions]
    #print("Mean")
    #pprint(mean)
   
    mean_new = [dict_mean(pred) for pred in predictions_new]
    #print("Mean New")
    #pprint(mean_new)
    
    
    res = np.argmax([get_improvement(mean[i], mean_new[i]) for i in range(len(mean))])
    
    print("argmax",res)
    return res
    
    # a good fit improves only one category and "worsens" the others 
    # take proba values into account
    
    #print(get_improvement(mean[0], mean_new[0]))
    
    # distinguishing factor: 
    # Rank 1 gets better, Other ones get worse
    
    # delta 
    
    
    # assume there is only one sequence where the prediction improves when adding the last element 


# Majority Voting by Stack
# Assign to Stack with biggest classification match
def classify_event_single(event): 
    print("frame number:",event["frame.number"])
    print([len(stack) for stack in stacks])
    # encode each event in the sequence
    sequences = [seq2features(stack + [event], bw=5) for stack in stacks]    
    
    print([len(seq) for seq in sequences])
    
    # for each element in the sequence we predict the activity
    seq_pred = [crf.predict_marginals_single(seq) for seq in sequences]
    
    # TODO evaluate 2 approaches - (1) mean then vote or (2) majority vote for each event
    
    # 1 
    seq_res = [dict_mean(pred[:-1]) for pred in seq_pred]
    pprint(seq_res)
    
    # classification of the new event 
    evt_res = [pred[-1] for pred in seq_pred]
    pprint(evt_res) 

    return 0

def search_stream_index(event): 
    stream_indices = [{e["stream_index"] for e in stack} for stack in stacks]
    stream_idx = event["stream_index"]
    # find stack indices that contain stream index of new event
    indices = [i for i, sublist in enumerate(stream_indices) if stream_idx in sublist]
    print("Stream indices",stream_idx,stream_indices, indices)
    if len(indices) == 1: 
        return indices[0]
    else:
        return 0
    
    
def classify_event_prob(event):
    # search for sequences in the training data
    # e.g. error threshold of n
    
    # list possible next events for both sequences 
    # compare to classify event
    pass 
    

def classify_event_markov(event):
    
    sequences = [[e["event_with_roles"]+e["selective_filter_data"] for e in stack]+[event["event_with_roles"]+event["selective_filter_data"]] for stack in stacks]
    probs = [mc.log_probability(seq) for seq in sequences]

    idx = np.argmax(probs)
    print("Classify Markov",probs, idx)
    return idx


def print_event(event): 
    keys = ["event_with_roles", "frame.number", "selective_filter_data", "request_method_call"]
    
    pprint({key: event[key] for key in keys})
    


# Event Loop Starts here #
verbose = False

stacks = []
res = []
stack_size = []

for i, event in enumerate(records):
    # TODO Preprocessing 
    # TODO Feature Extraction
    # TODO Activity Action Classification
    
    event_type = event["real_activity_action"]
    starting_frame = event["starting_frame_number"]

    if event_type == "Activity Start":
        if verbose: print(f"New\t\tStack\t\t\t{len(stacks)}", event["frame.number"])
        stacks.append([event])

    if event_type == "NoAction":
        if len(stacks) == 1:
            if verbose: print(f"Add\t\tBetween Event\t{0}")
            stacks[0].append(event)
        elif ~np.isnan(starting_frame):
            idx = search_stack(starting_frame)
            if verbose: print(f"Add\t\tBetween Event by request frame\t{idx}")
            stacks[idx].append(event)
        else:
            # Assign the event to one of the existing stacks by probability
            if verbose: print("Classify Between Event", event["frame.number"])
            stack_size.extend([len(stack) for stack in stacks])
            #print_event(event)
            #idx = search_stream_index(event)           
            
            #if idx == -1:
            idx = classify_by_train_sequences(event,4)
            
                #idx = classify_event_markov(event)   
                #idx = 0
                #idx = classify_event_by_information_gain(event)
            
            stacks[idx].append(event)
            
    if event_type == "Activity End":
        idx = search_stack(starting_frame)
        stacks[idx].append(event)
        stack = stacks.pop(idx)
        res.append(stack)
        if verbose: print(f"Close\tStack\t\t\t{idx}") 

event: 3126
[[50, 53, 44, 0], [1, 31, 0]]
res: [0, 18] 18 1 -> 1
event: 3131
[[50, 53, 44, 54], [1, 31, 0, 54]]
res: [4, 0] 4 1 -> 0
event: 3145
[[53, 44, 54, 42], [1, 31, 0, 42]]
res: [4, 0] 4 1 -> 0
event: 3193
[[44, 54, 42, 57], [1, 31, 0, 57]]
res: [4, 0] 4 1 -> 0
event: 3197
[[54, 42, 57, 42], [1, 31, 0, 42]]
res: [4, 0] 4 1 -> 0
event: 3217
[[42, 57, 42, 55], [1, 31, 0, 55]]
res: [0, 18] 18 1 -> 1
event: 3227
[[42, 57, 42, 57], [31, 0, 55, 57]]
res: [8, 0] 8 1 -> 0
event: 3233
[[57, 42, 57, 42], [31, 0, 55, 42]]
res: [4, 0] 4 1 -> 0
event: 3248
[[42, 57, 42, 16], [0, 55, 32, 16]]
res: [0, 7] 7 1 -> 1
event: 3258
[[42, 57, 42, 57], [55, 32, 16, 57]]
res: [8, 0] 8 1 -> 0
event: 3260
[[57, 42, 57, 46], [55, 32, 16, 46]]
res: [4, 0] 4 1 -> 0
event: 3262
[[42, 57, 46, 47], [55, 32, 16, 47]]
res: [4, 0] 4 1 -> 0
event: 3278
[[57, 46, 47, 63], [55, 32, 16, 63]]
res: [4, 0] 4 1 -> 0
event: 3287
[[46, 47, 63, 63], [55, 32, 16, 63]]
res: [4, 0] 4 1 -> 0
event: 3300
[[47, 63, 63, 63], [55, 

In [45]:
res_fn = [[e["frame.number"] for e in stack ]for stack in res]

In [48]:
pprint(res_fn)

[[17, 49, 57, 101, 110, 118, 175, 183, 241, 243, 245, 265, 291, 301, 325],
 [356,
  388,
  396,
  441,
  450,
  458,
  512,
  523,
  593,
  607,
  629,
  641,
  660,
  711,
  725,
  733,
  737,
  745,
  749,
  757,
  759,
  761,
  775,
  781,
  791,
  799,
  809,
  817,
  823,
  842,
  847,
  875,
  878,
  909,
  919,
  923,
  959,
  964,
  967,
  977,
  1034,
  1038,
  1058,
  1063,
  1065,
  1074,
  1113,
  1121,
  1185,
  1192],
 [1212,
  1244,
  1252,
  1296,
  1305,
  1313,
  1370,
  1378,
  1436,
  1438,
  1440,
  1460,
  1486,
  1497,
  1520],
 [1582,
  1614,
  1622,
  1665,
  1674,
  1682,
  1737,
  1747,
  1816,
  1830,
  1854,
  1865,
  1883,
  1933,
  1947,
  1956,
  1960,
  1968,
  1972,
  1980,
  1982,
  1984,
  1998,
  2004,
  2014,
  2022,
  2032,
  2040,
  2046,
  2065,
  2070,
  2098,
  2101,
  2132,
  2142,
  2146,
  2182,
  2187,
  2190,
  2200,
  2259,
  2263,
  2283,
  2288,
  2290,
  2299,
  2336],
 [2354,
  2387,
  2395,
  2439,
  2448,
  2456,
  2514,
  2522,
  

In [54]:
# Flatten the list of lists
flattened_list = [item for sublist in res_fn for item in sublist]

# Create a DataFrame with the flattened list and assign the index of each sublist to each row
df = pd.DataFrame({'Value': flattened_list})
df['stack_idx'] = [index for index, sublist in enumerate(res_fn) for _ in sublist]

In [60]:
def max_distance_between_occurrences(numbers):
    # Create a dictionary to store the last index where each number was seen
    last_seen = {}
    max_distances = {}

    for index, number in enumerate(numbers):
        if number in last_seen:
            # If the number has been seen before, update the maximum distance
            max_distances[number] = max(max_distances.get(number, 0), index - last_seen[number])

        # Update the last seen index for the current number
        last_seen[number] = index

    return max_distances

# Example usage:
numbers_list = [1, 2, 3,4, 1,1, 2, 1, 4, 5, 4]
result = max_distance_between_occurrences(numbers_list)

print(result)

{1: 4, 2: 5, 4: 5}


In [66]:
max_distance_between_occurrences(df.sort_values('Value')["stack_idx"].tolist())

{0: 1,
 1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 27,
 6: 20,
 7: 13,
 8: 12,
 9: 7,
 10: 1,
 11: 1,
 12: 1,
 13: 4,
 14: 5,
 15: 6,
 16: 4,
 17: 1,
 18: 11,
 20: 41,
 19: 12,
 21: 1,
 24: 9,
 22: 6,
 23: 8,
 25: 15,
 26: 8,
 28: 10,
 27: 6,
 29: 4,
 30: 4,
 31: 1,
 33: 18,
 32: 73,
 34: 37,
 35: 10,
 36: 1}

In [67]:
res_tuple = [(stack[0]["frame.number"], stack[-1]["frame.number"]) for stack in res]

In [69]:
from sklearn.metrics import classification_report,accuracy_score
# sort res
res = sorted(res_tuple, key= lambda x:x[0])

pred = [i for tup in res for i in tup]
true = df_gt[["start", "actual_end"]].values.flatten().tolist()
accuracy_score(true,pred)

1.0

In [350]:
start = [tup[0] for tup in res]
end = [tup[1] for tup in res]

res_df = pd.DataFrame({"start_pred":start, "end_pred":end})
res_df["start"] = df_gt["start"]
res_df["end"] = df_gt["actual_end"]

res_df["start_true"] = res_df["start"] == res_df["start_pred"]
res_df["end_true"] = res_df["end"] == res_df["end_pred"]

In [351]:
res_df

,start_pred,end_pred,start,end,start_true,end_true
0,17,325,17,325,True,True
1,356,1192,356,1192,True,True
2,1212,1520,1212,1520,True,True
3,1582,2336,1582,2336,True,True
4,2354,2664,2354,2664,True,True
5,2708,4461,2708,4461,True,True
6,3057,4871,3057,4871,True,True
7,4467,4881,4467,4881,True,True
8,4939,6164,4939,6164,True,True
9,5606,6859,5606,6859,True,True


In [250]:
df_gt

,activity_name,start,end,actual_end,overlapping_activities,bp_id,overlapping_bps,classification,Multi Class Classification,Single Class Classification,probability
0,GenerateJobApplicationActivity,17,356,325,[],1224,[],GenerateJobApplicationActivity,TRUE,True,0.994192
1,ResumeReviewActivity,356,1212,1192,[],1224,[],ResumeReviewActivity,TRUE,True,0.993149
2,GenerateJobApplicationActivity,1212,1582,1520,[],1225,[],GenerateJobApplicationActivity,TRUE,True,0.994192
3,ResumeReviewActivity,1582,2354,2336,[],1225,[],ResumeReviewActivity,TRUE,True,0.993329
4,GenerateJobApplicationActivity,2354,2708,2664,[],1226,[],GenerateJobApplicationActivity,TRUE,True,0.994192
5,ResumeReviewActivity,2708,4467,4461,['ScheduleAnInterviewActivityCall'],1226,[1255],"ResumeReviewActivity, ScheduleAnInterviewActiv...",TRUE,True,0.211112
6,ScheduleAnInterviewActivityCall,3057,4881,4871,"['ResumeReviewActivity', 'GenerateJobApplicati...",1225,"[1226,1227]","ResumeReviewActivity, ScheduleAnInterviewActiv...",TRUE,True,0.021792
7,GenerateJobApplicationActivity,4467,4939,4881,['ScheduleAnInterviewActivityCall'],1227,[1226],GenerateJobApplicationActivity,FALSE,True,0.986619
8,ResumeReviewActivity,4939,6205,6164,['PerformAnInterviewCall'],1227,[1225],ResumeReviewActivity,FALSE,True,0.839890
9,PerformAnInterviewCall,5606,6879,6859,['ResumeReviewActivity'],1225,[1227],PerformAnInterviewCall,FALSE,True,0.962106


In [219]:
df_gt[["start", "actual_end"]].values

array([[   17,   325],
       [  356,  1192],
       [ 1212,  1520],
       [ 1582,  2336],
       [ 2354,  2664],
       [ 2708,  4461],
       [ 3057,  4871],
       [ 4467,  4881],
       [ 4939,  6164],
       [ 5606,  6859],
       [ 6879,  7190],
       [ 7261,  8044],
       [ 8060,  8374],
       [ 8421,  9329],
       [ 9105, 10237],
       [10272, 10786],
       [10384, 11067],
       [11099, 11941],
       [11974, 14049],
       [12074, 14533],
       [13457, 14328],
       [14576, 15350],
       [15377, 17235],
       [15720, 16615],
       [16049, 17037],
       [17273, 18876],
       [17801, 19551],
       [19585, 20665],
       [20245, 20557],
       [20696, 21362],
       [20928, 21651],
       [21674, 22664],
       [22687, 26505],
       [22851, 25173],
       [23848, 27222],
       [25366, 27235],
       [27306, 28384]])